In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
pd.options.display.max_rows = 10
pd.options.display.max_columns = 500

In [2]:
###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['ml_outcomes'] = df['ml_outcomes_div'].round(2)


In [3]:
FRAC_ADMIT = df[['A']].sum()/len(df)


In [4]:
FRAC_ADMIT

A    0.485361
dtype: float64

In [5]:
len(df)

1000000

In [6]:
#df['ml_outcomes'] = df['R']

In [7]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_black','T_white','ml_outcomes_black','ml_outcomes_white','ml_outcomes_decision','B_p']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_black,T_white,ml_outcomes_black,ml_outcomes_white,ml_outcomes_decision,B_p
799856,0,7,0.02,6,7,-4.079949,-3.784626,0,0
381520,0,8,0.02,6,8,-4.079949,-3.705448,0,0
802371,0,12,0.03,10,12,-3.763237,-3.388736,0,0
475990,0,10,0.03,7,10,-4.000771,-3.547092,0,0
195611,0,12,0.03,7,12,-4.000771,-3.388736,0,0
...,...,...,...,...,...,...,...,...,...
774157,1,106,1.23,106,124,3.837851,5.479200,1,1
733667,1,108,1.23,108,126,3.996207,5.637556,1,1
777126,1,104,1.23,104,121,3.679495,5.241666,1,1
961510,1,105,1.23,105,123,3.758673,5.400022,1,1


In [8]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [9]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','B_p','ml_outcomes']].groupby(['R','T','B_p']).sum().reset_index()

In [10]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes','B_p']].groupby(['R','T','B_p']).count().reset_index()
df_count.columns = ['R','T','B_p','Count']
df_count['N'] = df_count['Count']

In [11]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [12]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,B_p,ml_outcomes,N
0,404,0,0,7,0,0.02,1
1,403,1,0,8,0,0.02,1
3,402,3,0,10,0,0.03,1
4,401,4,0,11,0,0.03,1
10,398,10,0,15,1,0.04,1
...,...,...,...,...,...,...,...
108,4,108,0,64,1,6614.24,9872
100,3,100,0,60,1,6639.00,11065
106,2,106,0,63,1,6648.84,10074
102,1,102,0,61,1,6774.12,10926


### Setup optimization problem 

In [13]:
from ortools.linear_solver import pywraplp


In [14]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [15]:
dff

,level_0,index,R,T,B_p,ml_outcomes,N
0,404,0,0,7,0,0.02,1
1,403,1,0,8,0,0.02,1
2,397,2,0,9,0,0.06,2
3,402,3,0,10,0,0.03,1
4,401,4,0,11,0,0.03,1
...,...,...,...,...,...,...,...
400,355,400,1,105,1,1.23,1
401,334,401,1,106,1,3.69,3
402,356,402,1,107,0,1.23,1
403,346,403,1,108,1,2.46,2


In [16]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'],row['B_p'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [17]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [18]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total applicants cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

485361


In [19]:
# Now we have one constraint
solver.NumConstraints()

1

## Add Equalized Odds Constraints

In [20]:
## Make sure that you have to add all people in B_p stratum or none
## i.e. you can't add only people who pass boards and reject those who fail boards from same T, R stratum
didntexist, exists = 0, 0 

for ix, row in dff.iterrows():
    constrain_bp = solver.Constraint(0.0, 0.0)
    
    var1 = vars_cache[(row['R'],row['T'],row['B_p'])]
    key2 = (row['R'],row['T'], 1-row['B_p'])
    
    if key2 not in vars_cache:
        didntexist+=1
        continue
        
    var2 = vars_cache[key2]
    
    constrain_bp.SetCoefficient(var1, -1.0)
    constrain_bp.SetCoefficient(var2, 1.0)
    exists+=1

didntexist, exists

(47, 358)

In [21]:
white_pass_boards = []
white_fail_boards = []
black_pass_boards = []
black_fail_boards = []

for key in vars_cache:
    r, t, b_p = key
    if b_p == 1 and r==0:
        white_pass_boards.append(key)
    elif b_p == 0 and r==0:
        white_fail_boards.append(key)
    elif b_p == 1 and r==1:
        black_pass_boards.append(key)
    elif b_p == 0 and r==1:
        black_fail_boards.append(key)

len(white_pass_boards),len(white_fail_boards),len(black_pass_boards),len(black_fail_boards)

(115, 101, 97, 92)

In [22]:
NUM_TOTALS = {}
df_totals = dff[['N','R','B_p']].groupby(['R','B_p']).sum().reset_index()
for ix, row in df_totals.iterrows():
    NUM_TOTALS[(row['R'],row['B_p'])] = row['N']
    
N_IN_STRATAS = {}
for ix, row in dff.iterrows():
    N_IN_STRATAS[(row['R'],row['T'],row['B_p'])] = row['N']

In [23]:
# Now we have one constraint
solver.NumConstraints()

406

In [24]:
#Of those who pass the boards exams
#Frac white admitted and frac black admitted should be the same

constrain_pass_boards = solver.Constraint(0.0, 0.0)

for key in white_pass_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_pass_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_pass_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


In [25]:
#Of those who fail the boards exams
#Frac white admitted and frac black admitted should be the same

constrain_fail_boards = solver.Constraint(0.0, 0.0)

for key in white_fail_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], float(N_IN_STRATUM) / float(N_TOTAL))

for key in black_fail_boards:
    r, t, b_p = key
    N_IN_STRATUM = N_IN_STRATAS[(r,t,b_p)]
    N_TOTAL = NUM_TOTALS[(r,b_p)]
    
    constrain_fail_boards.SetCoefficient(vars_cache[key], -1.0 * (float(N_IN_STRATUM) / float(N_TOTAL)))


## Solve linear program

In [26]:
solver.ABNORMAL

4

In [27]:
status = solver.Solve()


In [28]:
status

0

In [29]:
solver.OPTIMAL

0

In [30]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [31]:
df_decisions

,row_id,decision
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
400,400,1.0
401,401,1.0
402,402,1.0
403,403,1.0


In [32]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
104,0,104,0,62,1,6821.12,10658,104,1.0
102,1,102,0,61,1,6774.12,10926,102,1.0
106,2,106,0,63,1,6648.84,10074,106,1.0
100,3,100,0,60,1,6639.00,11065,100,1.0
108,4,108,0,64,1,6614.24,9872,108,1.0
...,...,...,...,...,...,...,...,...,...
12,400,12,0,16,1,0.04,1,12,0.0
4,401,4,0,11,0,0.03,1,4,0.0
3,402,3,0,10,0,0.03,1,3,0.0
1,403,1,0,8,0,0.02,1,1,0.0


In [33]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
216,392,216,1,4,0,0.26,1,216,0.0
0,404,0,0,7,0,0.02,1,0,0.0
217,385,217,1,7,0,0.54,2,217,0.0
1,403,1,0,8,0,0.02,1,1,0.0
218,391,218,1,8,0,0.27,1,218,0.0
...,...,...,...,...,...,...,...,...,...
211,368,211,0,125,1,1.00,1,211,1.0
212,369,212,0,127,1,1.00,1,212,1.0
213,370,213,0,128,1,1.00,1,213,1.0
214,371,214,0,130,1,1.00,1,214,1.0


In [34]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
104,0,104,0,62,1,6821.12,10658,104,1.0
102,1,102,0,61,1,6774.12,10926,102,1.0
106,2,106,0,63,1,6648.84,10074,106,1.0
100,3,100,0,60,1,6639.00,11065,100,1.0
108,4,108,0,64,1,6614.24,9872,108,1.0
...,...,...,...,...,...,...,...,...,...
12,400,12,0,16,1,0.04,1,12,0.0
4,401,4,0,11,0,0.03,1,4,0.0
3,402,3,0,10,0,0.03,1,3,0.0
1,403,1,0,8,0,0.02,1,1,0.0


In [35]:
xxx.to_csv('./decision.csv')

In [36]:
solution = pd.read_csv('./decision.csv')

In [37]:
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision
104,0,104,0,62,1,6821.12,10658,104,1.0
102,1,102,0,61,1,6774.12,10926,102,1.0
106,2,106,0,63,1,6648.84,10074,106,1.0
100,3,100,0,60,1,6639.00,11065,100,1.0
108,4,108,0,64,1,6614.24,9872,108,1.0
...,...,...,...,...,...,...,...,...,...
12,400,12,0,16,1,0.04,1,12,0.0
4,401,4,0,11,0,0.03,1,4,0.0
3,402,3,0,10,0,0.03,1,3,0.0
1,403,1,0,8,0,0.02,1,1,0.0


In [38]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)+'_'+df['B_p'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)+'_'+xxx['B_p'].astype(str)

In [39]:
xxx

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
104,0,104,0,62,1,6821.12,10658,104,1.0,0_62_1
102,1,102,0,61,1,6774.12,10926,102,1.0,0_61_1
106,2,106,0,63,1,6648.84,10074,106,1.0,0_63_1
100,3,100,0,60,1,6639.00,11065,100,1.0,0_60_1
108,4,108,0,64,1,6614.24,9872,108,1.0,0_64_1
...,...,...,...,...,...,...,...,...,...,...
12,400,12,0,16,1,0.04,1,12,0.0,0_16_1
4,401,4,0,11,0,0.03,1,4,0.0,0_11_0
3,402,3,0,10,0,0.03,1,3,0.0,0_10_0
1,403,1,0,8,0,0.02,1,1,0.0,0_8_0


In [40]:
admit_decisions = df.merge(xxx[['key','decision']],how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT['A']

FRAC_BLACK_POLICY = (admit_decisions['R'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['B_p'] * admit_decisions['decision_random']).sum()

In [41]:
file = open('./lp_results.csv','a')
file.write('{}\t{}\t{}\n'.format('Counterfactual Equalized Odds',str(FRAC_BLACK_POLICY),str(SUM_BP_POLICY)))
file.close()

In [42]:
xxx[xxx['R']==0].sort_values(by='decision')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
41,199,41,0,31,0,325.13,2501,41,0.0,0_31_0
29,259,29,0,25,0,55.44,616,29,0.0,0_25_0
31,252,31,0,26,0,67.23,747,31,0.0,0_26_0
44,249,44,0,32,1,72.94,521,44,0.0,0_32_1
33,241,33,0,27,0,99.50,995,33,0.0,0_27_0
...,...,...,...,...,...,...,...,...,...,...
130,53,130,0,75,1,3921.75,4725,130,1.0,0_75_1
180,227,180,0,100,1,138.71,143,180,1.0,0_100_1
82,51,82,0,51,1,3962.02,9214,82,1.0,0_51_1
103,55,103,0,62,0,3865.60,6040,103,1.0,0_62_0


In [43]:
xxx[xxx['R']==1].sort_values(by='decision')

,level_0,index,R,T,B_p,ml_outcomes,N,row_id,decision,key
286,27,286,1,44,0,4899.06,9606,286,0.0,1_44_0
257,242,257,1,29,1,98.60,290,257,0.0,1_29_1
240,239,240,1,21,0,103.20,344,240,0.0,1_21_0
259,232,259,1,30,1,127.05,363,259,0.0,1_30_1
242,225,242,1,22,0,146.32,472,242,0.0,1_22_0
...,...,...,...,...,...,...,...,...,...,...
359,211,359,1,80,1,230.88,208,359,1.0,1_80_1
361,208,361,1,81,1,240.80,215,361,1.0,1_81_1
342,206,342,1,72,0,244.42,242,342,1.0,1_72_0
371,235,371,1,86,1,107.88,93,371,1.0,1_86_1


In [44]:
(xxx['N']*xxx['decision']).sum()/len(df)

0.48536100000000054

In [45]:
admit_decisions[admit_decisions['B_p']==1][['decision','R']].groupby(['R']).mean()

,decision
R,
0,0.663304
1,0.663304
